In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
def load_close_prices(tickers):
#     prices are not adjusted for splits or dividends...
    history = {
        tic: yf.Ticker(tic).history(period='max')
        for tic in tickers}
    indexes = [set(data.index) for data in history.values()]
    index = sorted(set.union(*indexes))
    closes = pd.concat([
        history[tic].reindex(index=index)['Close'].ffill()
        for tic in tickers], axis=1)
    closes.columns = tickers
    stacked = closes.stack().reset_index()
    stacked.columns = ['date', 'ticker', 'price']
#     stacked.sort_values('date', inplace=True)
    return stacked

In [ ]:
def compute_pnl(frame):
    frame['market_value'] = frame['price'] * frame['position']
    frame['cash'] = -frame['position'].diff() * frame['price']
    first_date = min(frame.index)
    frame.loc[first_date, 'cash'] = -frame.loc[first_date, 'market_value']
    frame['cash'] = frame['cash'].cumsum()
    frame['nav'] = frame['market_value'] + frame['cash']
    return frame

In [ ]:
%%time
tickers = ['msft', 'aapl', 'goog']
closes = load_close_prices(tickers)
closes

In [ ]:
closes['position'] = 2
closes.iloc[:len(closes) // 2]['position'] = 1

In [ ]:
navs = (
    closes
    .set_index('date')
    .groupby('ticker')
    .apply(compute_pnl)
    .reset_index())
ptfo = navs.groupby('date').sum()
ptfo['nav'].plot()

In [ ]:
navs[
    (navs['ticker'] == 'aapl') & (navs['position'].diff() != 0)]

In [ ]:
navs[(navs['ticker'] == 'aapl') & (navs['date'] >= '2006-04-20')]